In [1]:
import pandas as pd

# Daily infections number

In [2]:
county_daily_urls = ['https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-2020.csv',
                     'https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-2021.csv',
                     'https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-2022.csv',
                     'https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-recent.csv'
                    ]

In [3]:
def get_county_daily_infections_nyt(urls):
    county_dfs = [pd.read_csv(url) for url in urls]
    df = pd.concat(county_dfs, ignore_index=True)
    # Column fips
    df. rename(columns = {'geoid':'fips'}, inplace = True)
    df['fips'] = df['fips'].str[4:]
    # Removing 2 columns
    df = df.drop(['cases_avg_per_100k', 'deaths_avg_per_100k'], axis = 1)
    # Deleteing data for "Unknown" county 
    df = df.query('county != "Unknown"')
    return df

In [ ]:
county_daily_df = get_county_daily_infections_nyt(county_daily_urls)

In [ ]:
county_daily_df

In [ ]:
# Grouped by FIPS
# county_grouped = county_df.groupby(county_daily_df.fips)

# Cumulative infections

In [ ]:
county_cum_urls = ['https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2020.csv',
                   'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2021.csv',
                   'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2022.csv',
                   'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-recent.csv'
                  ]

In [ ]:
def get_county_cum_infections_nyt(urls):
    county_dfs = [pd.read_csv(url) for url in urls]
    df = pd.concat(county_dfs, ignore_index=True)
    # Deleteing data for "Unknown" county
    df = df.query('county != "Unknown"')
    # Fill NaN values
    m_d = (df['deaths'].isnull() == True)
    m_c = (df['cases'].isnull() == True)
    df.loc[m_d, 'deaths'] = 0
    df.loc[m_c, 'cases'] = 0
    # New York City
    # All cases for the five boroughs of New York City (New York, Kings, Queens, Bronx and Richmond counties)
    # are assigned to a single area called New York City. The number of deaths in New York City also includes
    # probable deaths reported by the New York City health department.
    # Deaths are reported by county of residence, except for certain periods described below.
    # FIPS code for New York State
    m_NY = (df['county'] == "New York City") & (df['fips'].isnull() == True)
    df.loc[m_NY, 'fips'] = 36
    # Kansas City
    # Four counties (Cass, Clay, Jackson and Platte) overlap the municipality of Kansas City, Mo.
    # The cases and deaths that we show for these four counties are only for the portions exclusive of Kansas City.
    # Cases and deaths for Kansas City are reported as their own line.
    # FIPS code for Kansas State
    m_KC = (df['county'] == "Kansas City") & (df['fips'].isnull() == True)
    df.loc[m_KC, 'fips'] = 20
    # City Joplin
    # FIPS code for the city
    m_J = (df['county'] == "Joplin") & (df['fips'].isnull() == True)
    df.loc[m_J, 'fips'] = 2937592
    # FIPS code - change from .0
    df = df.astype({"fips": int, "deaths": int})
    return df

In [ ]:
county_cum_df = get_county_cum_infections_nyt(county_cum_urls)

In [ ]:
county_cum_df